# 快速开始

在本教程中，我们假设 Bridgic 已经安装在您的系统上。如果不是，请参见 [安装指南](../../../installation)。

让我们开始构建一个简单的单词学习助手。您提供一个单词，助手将生成其派生形式并用它们创建句子。这个例子还将展示如何在实践中使用 Bridgic。

## 单词学习助手

### 1. 模型初始化

在开始之前，让我们设置我们的环境。在这个快速开始中，我们将直接使用集成。有关模型集成的深入解释，请参见：[LLM 集成](../../model_integration/llm_integration/)。

执行以下命令在 shell 中：

```shell
export OPENAI_API_KEY="<your_openai_api_key>"
export OPENAI_MODEL_NAME="<the_model_name>"
```

In [ ]:
# Get the environment variables.
import os

# Import the necessary packages.
from bridgic.core.automa import GraphAutoma, worker
from bridgic.core.model.types import Message, Role

# Here we use OpenAILikeLlm because the package `bridgic-llms-openai-like` is installed automatically 
# when you install Bridgic. This makes sure the OpenAI-like model integration works out of the box.
from bridgic.llms.openai_like import OpenAILikeLlm, OpenAILikeConfiguration


# In this tutorial, we use OpenAI as an example. 
# You can freely replace these model settings to use any LLM provider you like.
_api_key = os.environ.get("OPENAI_API_KEY")
_api_base = os.environ.get("OPENAI_API_BASE")
_model_name = os.environ.get("OPENAI_MODEL_NAME")

llm = OpenAILikeLlm(
    api_key=_api_key,
    api_base=_api_base,
    configuration=OpenAILikeConfiguration(model=_model_name),
    timeout=20,
)

### 2. Automa 编排

完成单词学习助手有两个步骤：

1. 生成输入单词的衍生词。
2. 用衍生词造句。

In [19]:
class WordLearningAssistant(GraphAutoma):
    @worker(is_start=True)
    async def generate_derivatives(self, word: str):
        print(f"------Generating derivatives for {word}------")
        response = await llm.achat(
            model=_model_name,
            messages=[
                Message.from_text(text="You are a word learning assistant. Generate derivatives of the input word in a list.", role=Role.SYSTEM),
                Message.from_text(text=word, role=Role.USER),
            ]
        )
        print(response.message.content)
        print(f"------End of generating derivatives------\n")
        return response.message.content

    @worker(dependencies=["generate_derivatives"], is_output=True)
    async def make_sentences(self, derivatives):
        print(f"------Making sentences with------")
        response = await llm.achat(
            model=_model_name,
            messages=[
                Message.from_text(text="You are a word learning assistant. Make sentences with the input derivatives in a list.", role=Role.SYSTEM),
                Message.from_text(text=derivatives, role=Role.USER),
            ]
        )
        print(response.message.content)
        print(f"------End of making sentences------\n")
        return response.message.content

word_learning_assistant = WordLearningAssistant()

### 3. 代理运行

让我们通过 [`arun`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.GraphAutoma.arun) 方法运行这个助手：

In [20]:
res = await word_learning_assistant.arun(word="happy")

------Generating derivatives for happy------
Here are some derivatives of the word "happy":

1. Happiness
2. Happily
3. Happier
4. Happiest
5. Unhappy
6. Unhappiness
7. Happinesses (plural form)
8. Happifying (gerund form)
9. Happify (verb form)

Feel free to ask for derivatives of another word!
------End of generating derivatives------

------Making sentences with------
Sure! Here are sentences using each of the derivatives of the word "happy":

1. **Happiness**: The pursuit of happiness is a common goal for many people.
2. **Happily**: She smiled happily as she opened her birthday gifts.
3. **Happier**: After taking a vacation, I felt much happier than I had in months.
4. **Happiest**: That day was the happiest moment of my life when my daughter graduated.
5. **Unhappy**: He seemed unhappy at the party and left early.
6. **Unhappiness**: Her unhappiness was evident in her quiet demeanor.
7. **Happinesses**: Different people find happinesses in various aspects of life, like family, wo

恭喜！我们已经成功完成了单词学习助手，它完全按照我们的要求执行了任务。

<div style="text-align: center; margin: 2rem 0;">
<hr style="border: none; border-top: 2px solid #e2e8f0;">
</div>

## 我们学到了什么？

上述示例展示了使用 Bridgic 编写代理应用程序的典型方式。现在让我们探索它的一些组件。

### Worker

任何可调用对象（例如函数、方法等）都可以转换为 Worker 对象，作为 Bridgic 中调度和编排的 **基本执行单元**。

就像单词学习助手的示例一样，我们可以使用装饰器语法 [`@worker`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.worker._worker_decorator.worker) 将函数和方法包装成 Worker 对象。

In [ ]:
class MyAutoma(GraphAutoma):
    @worker(is_start=True)
    async def start(self, x: int):
        return x    

或者，您也可以这样使用它：

In [ ]:
class MyAutoma(GraphAutoma): ...
my_automa = MyAutoma()

# Add the function as a worker with worker decorator in the instance of the automa
@my_automa.worker(is_start=True)
async def start(x: int):
    return x


另一个 API [`add_func_as_worker()`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.GraphAutoma.add_func_as_worker) 也可以用来将 Worker 添加到 [`GraphAutoma`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.GraphAutoma) 中。

In [ ]:
async def start(x: int):
    return x

class MyAutoma(GraphAutoma): ...
my_automa = MyAutoma()

# Add the function as a worker
my_automa.add_func_as_worker(
    key="start",
    func=start,
    is_start=True,
)


除了可以转换为 Worker 的函数之外，继承自 [`Worker`](../../../../reference/bridgic-core/bridgic/core/automa/worker/#bridgic.core.automa.worker.Worker) 并重写 [`run()`](../../../../reference/bridgic-core/bridgic/core/automa/worker/#bridgic.core.automa.worker.Worker.run) 或 [`arun()`](../../../../reference/bridgic-core/bridgic/core/automa/worker/#bridgic.core.automa.worker.Worker.arun) 的子类也可以直接用作 Worker，其实例可以通过 [`add_worker()`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.GraphAutoma.add_worker) API 添加到 `GraphAutoma` 中。

In [ ]:
from bridgic.core.automa.worker import Worker

class MyWorker(Worker):
    async def arun(self, x: int):
        return x

my_worker = MyWorker()

# Add the worker to the automa
class MyAutoma(GraphAutoma): ...
my_automa = MyAutoma()
my_automa.add_worker(
    key="my_worker",
    worker=my_worker,
    is_start=True,
)

# Run the worker
res = await my_automa.arun(x=1)
print(res)

> 注意：
> 1. 继承自 `Worker` 的特定 worker 必须重写 `run()` 或 `arun()` 方法中的一个。
> 2. Bridgic 是一个主要设计用于异步执行的框架，如果 worker 的 `run()` 和 `arun()` 都被重写，则 `arun()` 将优先执行。有关详细信息，请参阅 [`Worker`](../../../../reference/bridgic-core/bridgic/core/automa/worker/#bridgic.core.automa.worker.Worker)。

以任何这些方式，workers 都可以正确地添加到 `MyAutoma` 中。

无论使用装饰器语法还是相应的 API，通常都有一些参数：

1. `key`：用作 worker 键的字符串。作为当前 automa 中 worker 的唯一标识符，必须确保在同一 automa 中没有重复的键。默认使用函数或类名。
2. `func`（在 [`add_func_as_worker()`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.GraphAutoma.add_func_as_worker) 中）或 `worker`（在 [`add_worker()`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.GraphAutoma.add_worker) 中）：实际可调用对象。**装饰器语法不需要此参数。**
3. `is_start`：`True` 或 `False`。将 worker 标记为 automa 的起始 worker。可以为多个 workers 设置。
4. `dependencies`：worker 键的列表。标记该 worker 依赖的前置 workers。
5. `is_output`：`True` 或 `False`。将 worker 标记为 automa 的输出 worker。每个执行分支只能设置一个输出 worker。
6. `args_mapping_rule`：[参数映射规则](../../../../reference/bridgic-core/bridgic/core/automa/args/#bridgic.core.automa.args.ArgsMappingRule)。有关 workers 之间参数绑定的详细信息，请参阅教程：[参数绑定](../../core_mechanism/parameter_binding/)

> 注意：在 Bridgic 中，worker 必须在调度和执行之前添加到 automa。换句话说，您不应该直接调用 `worker.arun()` 或 `worker.run()` 来运行 worker。

### GraphAutoma

一个 automa 是管理和协调一组 workers 的实体，充当 **调度引擎**。在上面的单词学习助手示例中，我们使用了 [`Automa`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.Automa) 的子类，即 [`GraphAutoma`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.GraphAutoma)，它根据 workers 之间的拓扑排序执行调度。

您应该子类化 `GraphAutoma` 并使用 [`@worker`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.worker._worker_decorator.worker) 声明方法作为 workers。

In [3]:
# Write workers in MyAutoma
class MyAutoma(GraphAutoma):
    @worker(is_start=True)
    async def worker_0(self, a, b, x, y):
        print(f"worker_0: a={a}, b={b}, x={x}, y={y}")

    @worker(is_start=True)
    async def worker_1(self, x, y):
        print(f"worker_1: x={x}, y={y}")

在定义完所有必需的 Worker 后，可以通过 `await automa_obj.arun(*args, **kwargs)` 来调用 automa，启动整个调度过程。

> Bridgic 是一个基于异步编程的框架。因此，必须使用 arun() 启动 `Graphautoma`。但是，Worker 可以在需要时以 [并发模式](../../../../tutorials/items/core_mechanism/concurrency_mode/) 执行。

在启动时，`automa_obj.arun(*args, **kwargs)` 的参数将根据位置参数和关键字参数分配给标记为 `is_start=True` 的 Worker。

- 位置参数：传递给 `arun` 的位置参数映射到标记为 `is_start=True` 的 Worker 的参数，遵循提供的顺序。如果某个 Worker 的参数列表短于传递给 `arun()` 的位置参数数量，将会引发错误。
- 关键字参数：传递给 `arun` 的关键字参数映射到标记为 `is_start=True` 的 Worker 的相应参数。
- 优先级：**位置参数优先于关键字参数。**

例如：我们传递位置参数 `1` 和 `2`，以及关键字参数 `x=3`，`y=4`。

In [4]:
my_automa = MyAutoma()
await my_automa.arun(1, 2, x=3, y=4)

worker_0: a=1, b=2, x=3, y=4
worker_1: x=1, y=2


`1` 和 `2` 分别被 `worker_0` 和 `worker_1` 的第一个和第二个参数按顺序接收。因为位置参数优先于关键字参数，即使 `worker_1` 的参数名称与输入的关键字参数相同，它们仍然会优先接收位置参数。

如果某个 worker 的参数列表短于传递给 `arun` 的位置参数数量，将会引发错误。

In [ ]:
my_automa = MyAutoma()
await my_automa.arun(1, 2, 3, y=4)  # worker_1 raises an error


如果所有参数以关键字格式传递，则每个 `is_start=True` 的 Worker 可以接收相应的值。

In [10]:
my_automa = MyAutoma()
await my_automa.arun(a=1, b=2, x=3, y=4)

worker_0: a=1, b=2, x=3, y=4
worker_1: x=3, y=4


现在，我们可以开始构建我们的 Bridgic 项目！